# Phase 3: Hybrid Search - Educational Walkthrough

This notebook walks through Phase 3 implementation step-by-step.

**What you'll learn:**
1. What sparse vectors are and why they matter
2. How to generate BM25-style sparse vectors
3. How Reciprocal Rank Fusion (RRF) combines dense + sparse results
4. How to implement payload filtering in Qdrant
5. How to create time-aware queries (upcoming events, recent blogs)
6. Why we store dates as timestamps for filtering

**Prerequisites:**
- Phase 2 complete (multi-content storage)
- Qdrant running in Docker
- Understanding of dense vector search

## 📚 Background: Dense vs. Sparse Vectors

### Dense Vectors (Phase 1 & 2)
- **What:** 384-dimensional embeddings from sentence-transformers
- **Strength:** Captures semantic meaning ("ML workshop" ≈ "machine learning seminar")
- **Weakness:** Can miss exact keyword matches

### Sparse Vectors (Phase 3 - New!)
- **What:** High-dimensional vectors with mostly zeros, only non-zero for present words
- **Strength:** Excellent for exact keyword matching (BM25-style)
- **Weakness:** Misses semantic similarity

### Hybrid = Best of Both Worlds
Combine dense (semantic) + sparse (keyword) for superior search quality!

## Setup: Import Libraries and Initialize

In [1]:
import sys, os
print("🟢 Python executable:", sys.executable)
print("🟢 venv prefix:", sys.prefix)
print("🟢 $VIRTUAL_ENV:", os.getenv("VIRTUAL_ENV"))

🟢 Python executable: /Users/Shared/ALL WORKSPACE/Hackathons/mom_hack/.venv/bin/python
🟢 venv prefix: /Users/Shared/ALL WORKSPACE/Hackathons/mom_hack/.venv
🟢 $VIRTUAL_ENV: /Users/Shared/ALL WORKSPACE/Hackathons/mom_hack/.venv


In [2]:
# Standard library imports
import sys
import os
from pathlib import Path
from datetime import datetime, timedelta
import time

# Setup project root
project_root = Path.cwd().parent.absolute()
os.chdir(project_root)

if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

# Our custom modules
from app.database.qdrant_client import QdrantService
from app.services.embedder import EmbeddingService
from app.utils.sparse_vector import create_sparse_vector
from qdrant_client.models import PointStruct, SparseVector

print("✅ All imports successful!")
print(f"📁 Working directory: {os.getcwd()}")

/Users/Shared/ALL WORKSPACE/Hackathons/mom_hack/.venv/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


✅ All imports successful!
📁 Working directory: /Users/Shared/ALL WORKSPACE/Hackathons/mom_hack/feedprism_main


## Step 1: Understanding Sparse Vectors

**Concept:** Sparse vectors represent text using word counts, similar to BM25 ranking.

**Format:**
```python
{
    "indices": [123, 456, 789],  # Hashed word IDs
    "values": [2.0, 1.0, 3.0]     # Word frequencies
}
```

Let's see how this works:

In [3]:
# Test sparse vector generation
text1 = "Machine learning workshop on deep learning"
text2 = "Python programming course"

sparse1 = create_sparse_vector(text1)
sparse2 = create_sparse_vector(text2)

print("🔤 Sparse Vector for:", text1)
print(f"   Indices: {sparse1['indices'][:5]}...  (showing first 5)")
print(f"   Values:  {sparse1['values'][:5]}...")
print(f"   Total unique words: {len(sparse1['indices'])}")

print("\n🔤 Sparse Vector for:", text2)
print(f"   Indices: {sparse2['indices'][:5]}...")
print(f"   Values:  {sparse2['values'][:5]}...")
print(f"   Total unique words: {len(sparse2['indices'])}")

print("\n💡 Key insight: 'learning' appears twice in text1, so its value is 2.0")

🔤 Sparse Vector for: Machine learning workshop on deep learning
   Indices: [733227, 5473, 968525, 902716, 747545]...  (showing first 5)
   Values:  [1.0, 2.0, 1.0, 1.0, 1.0]...
   Total unique words: 5

🔤 Sparse Vector for: Python programming course
   Indices: [140964, 574864, 241077]...
   Values:  [1.0, 1.0, 1.0]...
   Total unique words: 3

💡 Key insight: 'learning' appears twice in text1, so its value is 2.0


## Step 2: Creating a Collection with Sparse Vector Support

**What's new:** We must configure Qdrant to accept both dense and sparse vectors.

**Configuration:**
```python
vectors_config=VectorParams(size=384, distance=COSINE),  # Dense
sparse_vectors_config={"keywords": SparseVectorParams()} # Sparse
```

In [4]:
# Initialize services
print("🚀 Initializing services...\n")
qdrant = QdrantService()
embedder = EmbeddingService()

# Create collection with sparse vector support
print("📦 Creating collection with hybrid vector support...")
qdrant.create_collection(recreate=True)

# Verify collection was created
info = qdrant.get_collection_info()
print(f"\n✅ Collection '{info['name']}' created")
print(f"   Status: {info['status']}")

2025-11-27 09:58:11.622 | INFO     | app.database.qdrant_client:__init__:92 - Connecting to Qdrant: localhost:6333
2025-11-27 09:58:11.669 | SUCCESS  | app.database.qdrant_client:__init__:94 - Qdrant client initialized
2025-11-27 09:58:11.669 | INFO     | app.services.embedder:__init__:27 - Loading embedding model: sentence-transformers/all-MiniLM-L6-v2


🚀 Initializing services...



2025-11-27 09:58:15.061 | SUCCESS  | app.services.embedder:__init__:29 - Model loaded: 384D vectors
2025-11-27 09:58:15.223 | WARNING  | app.database.qdrant_client:create_collection:104 - Deleting existing collection: feedprism_emails


📦 Creating collection with hybrid vector support...


2025-11-27 09:58:15.310 | INFO     | app.database.qdrant_client:create_collection:108 - Creating collection: feedprism_emails
2025-11-27 09:58:15.627 | SUCCESS  | app.database.qdrant_client:create_collection:121 - Collection created: feedprism_emails



✅ Collection 'feedprism_emails' created
   Status: green


## Step 3: Upserting Points with Both Dense and Sparse Vectors

**Key change:** Each point now has TWO vectors:
- `""` (empty string) → Dense vector (384D)
- `"keywords"` → Sparse vector (variable dimension)

**Date storage:** We store dates as **timestamps** (floats) for range filtering.

Why timestamps?
- Qdrant's `Range` filter requires numeric values
- ISO strings can't be compared numerically
- We keep `start_date_iso` for display

In [5]:
# Prepare test data
print("📝 Preparing hybrid vector data...\n")

# Events (upcoming)
events = [
    {
        "text": "Machine Learning Workshop: Deep Dive into Transformers",
        "type": "event",
        "date": (datetime.now() + timedelta(days=5)).isoformat(),
        "tags": ["ai", "ml", "workshop"]
    },
    {
        "text": "Web Development Bootcamp: React and Node.js",
        "type": "event",
        "date": (datetime.now() + timedelta(days=15)).isoformat(),
        "tags": ["web", "javascript", "bootcamp"]
    },
    {
        "text": "Data Science Conference 2025",
        "type": "event",
        "date": (datetime.now() + timedelta(days=45)).isoformat(),  # Too far
        "tags": ["data", "science", "conference"]
    }
]

# Blogs (recent)
blogs = [
    {
        "text": "Understanding Sparse Vectors in Qdrant",
        "type": "blog",
        "date": (datetime.now() - timedelta(days=2)).isoformat(),  # Recent
        "tags": ["qdrant", "vector-db", "tutorial"]
    },
    {
        "text": "The Future of AI Agents",
        "type": "blog",
        "date": (datetime.now() - timedelta(days=10)).isoformat(),  # Too old
        "tags": ["ai", "agents", "future"]
    }
]

all_items = events + blogs
points = []

for i, item in enumerate(all_items):
    # Generate DENSE vector
    dense_vec = embedder.embed_text(item["text"])
    
    # Generate SPARSE vector
    sparse_vec_dict = create_sparse_vector(item["text"])
    sparse_vec = SparseVector(**sparse_vec_dict)
    
    # Convert date to timestamp (REQUIRED for Range filtering)
    date_timestamp = datetime.fromisoformat(item["date"]).timestamp()
    
    # Create point with BOTH vectors
    vector = {
        "": dense_vec,           # Dense vector (default)
        "keywords": sparse_vec   # Sparse vector (named)
    }
    
    point = PointStruct(
        id=i + 1,
        vector=vector,
        payload={
            "content_type": item["type"],
            "title": item["text"],
            "start_date": date_timestamp,     # Numeric for filtering
            "start_date_iso": item["date"],   # String for display
            "tags": item["tags"]
        }
    )
    points.append(point)
    
    print(f"{i+1}. {item['type']:6s} | {item['text'][:50]:50s} | {item['date'][:10]}")

# Upsert to Qdrant
print(f"\n📤 Upserting {len(points)} points with hybrid vectors...")
qdrant.upsert_points(points)

# Wait for indexing
time.sleep(2)
print("\n✅ Data indexed!")

📝 Preparing hybrid vector data...

1. event  | Machine Learning Workshop: Deep Dive into Transfor | 2025-12-02
2. event  | Web Development Bootcamp: React and Node.js        | 2025-12-12
3. event  | Data Science Conference 2025                       | 2026-01-11


2025-11-27 10:04:33.355 | INFO     | app.database.qdrant_client:upsert_points:136 - Upserting 5 points to feedprism_emails


4. blog   | Understanding Sparse Vectors in Qdrant             | 2025-11-25
5. blog   | The Future of AI Agents                            | 2025-11-17

📤 Upserting 5 points with hybrid vectors...


2025-11-27 10:04:33.545 | SUCCESS  | app.database.qdrant_client:upsert_points:143 - Upserted 5 points



✅ Data indexed!


## Step 4: Hybrid Search with RRF Fusion

**How it works:**
1. Run TWO separate searches: dense + sparse
2. Each returns ranked results
3. Combine using Reciprocal Rank Fusion (RRF):
   ```
   score(doc) = Σ 1/(k + rank_i)
   ```
   where `k=60` is a constant, `rank_i` is position in each result list

**Why RRF?**
- No need to normalize scores from different systems
- Robust to outliers
- Simple and effective

In [6]:
# Test hybrid search
query = "machine learning"
query_vec = embedder.embed_text(query)

print(f"🔍 Hybrid Search Query: '{query}'\n")
print("How it works:")
print("  1. Dense search finds semantically similar docs")
print("  2. Sparse search finds keyword matches")
print("  3. RRF combines rankings\n")

results = qdrant.hybrid_search(query_vec, query, limit=5)

print("📊 Results:\n")
for i, res in enumerate(results, 1):
    content_type = res['payload']['content_type']
    title = res['payload']['title']
    print(f"{i}. [{content_type:5s}] {title}")

print("\n💡 Notice: 'Machine Learning Workshop' ranked #1")
print("   - Dense: High semantic similarity")
print("   - Sparse: Exact 'machine' and 'learning' keyword matches")
print("   - RRF combines both signals → top result!")

🔍 Hybrid Search Query: 'machine learning'

How it works:
  1. Dense search finds semantically similar docs
  2. Sparse search finds keyword matches
  3. RRF combines rankings

📊 Results:

1. [event] Machine Learning Workshop: Deep Dive into Transformers
2. [event] Data Science Conference 2025
3. [blog ] The Future of AI Agents
4. [blog ] Understanding Sparse Vectors in Qdrant
5. [event] Web Development Bootcamp: React and Node.js

💡 Notice: 'Machine Learning Workshop' ranked #1
   - Dense: High semantic similarity
   - Sparse: Exact 'machine' and 'learning' keyword matches
   - RRF combines both signals → top result!


## Step 5: Payload Filtering by Content Type

**Use case:** Search only within events OR blogs.

**Implementation:** Qdrant's `Filter` with `FieldCondition`

In [7]:
# Search with content type filter
print("🔍 Filtered Search: Events only\n")

event_results = qdrant.search_with_filters(
    query_vector=query_vec,
    content_type="event",  # Filter by type
    limit=5
)

print("📊 Event Results:\n")
for i, res in enumerate(event_results, 1):
    title = res['payload']['title']
    date = res['payload']['start_date_iso'][:10]
    print(f"{i}. {title} ({date})")

print(f"\n✅ Found {len(event_results)} events (no blogs)")

🔍 Filtered Search: Events only

📊 Event Results:

1. Data Science Conference 2025 (2026-01-11)
2. Machine Learning Workshop: Deep Dive into Transformers (2025-12-02)
3. Web Development Bootcamp: React and Node.js (2025-12-12)

✅ Found 3 events (no blogs)


## Step 6: Time-Aware Queries - Upcoming Events

**Goal:** Find events happening in the next 30 days.

**Challenge:** Qdrant's `Range` filter only works with numeric values.

**Solution:**
1. Store dates as Unix timestamps (floats)
2. Create range: `[now_timestamp, future_timestamp]`
3. Filter using numeric comparison

In [8]:
# Search for upcoming events
print("🔍 Time-Aware Search: Upcoming Events (Next 30 days)\n")

# This method:
# 1. Creates date range: today -> +30 days
# 2. Converts ISO strings to timestamps
# 3. Applies Range filter on start_date field
# 4. Also filters by content_type="event"

upcoming = qdrant.search_upcoming_events(
    query_vector=query_vec,
    days_ahead=30,
    limit=5
)

print("📅 Upcoming Events:\n")
for i, res in enumerate(upcoming, 1):
    title = res['payload']['title']
    date_iso = res['payload']['start_date_iso']
    date_str = date_iso[:10]
    
    # Calculate days from now
    event_dt = datetime.fromisoformat(date_iso)
    days_away = (event_dt - datetime.now()).days
    
    print(f"{i}. {title}")
    print(f"   📆 {date_str} (in {days_away} days)\n")

print(f"✅ Found {len(upcoming)} upcoming events")
print("\n💡 Events more than 30 days away are filtered out!")
print("   (Data Science Conference is 45 days away → excluded)")

🔍 Time-Aware Search: Upcoming Events (Next 30 days)

📅 Upcoming Events:

1. Machine Learning Workshop: Deep Dive into Transformers
   📆 2025-12-02 (in 4 days)

2. Web Development Bootcamp: React and Node.js
   📆 2025-12-12 (in 14 days)

✅ Found 2 upcoming events

💡 Events more than 30 days away are filtered out!
   (Data Science Conference is 45 days away → excluded)


## Step 7: Time-Aware Queries - Recent Blogs

**Goal:** Find blog posts from the last 7 days.

**Same principle, opposite direction:**
- Range: `[past_timestamp, now_timestamp]`
- Filter: `content_type="blog"`

In [9]:
# Search for recent blogs
print("🔍 Time-Aware Search: Recent Blogs (Last 7 days)\n")

recent_blogs = qdrant.search_recent_blogs(
    query_vector=query_vec,
    days_back=7,
    limit=5
)

print("📰 Recent Blogs:\n")
for i, res in enumerate(recent_blogs, 1):
    title = res['payload']['title']
    date_iso = res['payload']['start_date_iso']
    date_str = date_iso[:10]
    
    # Calculate days ago
    blog_dt = datetime.fromisoformat(date_iso)
    days_ago = (datetime.now() - blog_dt).days
    
    print(f"{i}. {title}")
    print(f"   📆 {date_str} ({days_ago} days ago)\n")

print(f"✅ Found {len(recent_blogs)} recent blogs")
print("\n💡 Blogs older than 7 days are filtered out!")
print("   ('The Future of AI Agents' is 10 days old → excluded)")

🔍 Time-Aware Search: Recent Blogs (Last 7 days)

📰 Recent Blogs:

1. Understanding Sparse Vectors in Qdrant
   📆 2025-11-25 (2 days ago)

✅ Found 1 recent blogs

💡 Blogs older than 7 days are filtered out!
   ('The Future of AI Agents' is 10 days old → excluded)


## Step 8: Comparing Search Modes

Let's run all three search modes side-by-side to see the differences:

In [10]:
# Comparison test
query = "transformers"
query_vec = embedder.embed_text(query)

print(f"🔍 Query: '{query}'\n")
print("="*70)

# 1. Dense-only search (legacy)
print("\n1️⃣ Dense-Only Search (Phase 1 style):")
dense_results = qdrant.search(query_vec, limit=3)
for i, res in enumerate(dense_results, 1):
    print(f"   {i}. {res['payload']['title'][:60]} (score: {res['score']:.3f})")

# 2. Hybrid search (Phase 3)
print("\n2️⃣ Hybrid Search (Dense + Sparse + RRF):")
hybrid_results = qdrant.hybrid_search(query_vec, query, limit=3)
for i, res in enumerate(hybrid_results, 1):
    print(f"   {i}. {res['payload']['title'][:60]}")

# 3. Hybrid + Filtering
print("\n3️⃣ Hybrid + Time Filter (Upcoming Events):")
filtered_results = qdrant.search_upcoming_events(query_vec, days_ahead=30, limit=3)
for i, res in enumerate(filtered_results, 1):
    date = res['payload']['start_date_iso'][:10]
    print(f"   {i}. {res['payload']['title'][:50]} ({date})")

print("\n" + "="*70)
print("\n💡 Key Insights:")
print("   - Dense search: Good semantic understanding")
print("   - Hybrid: Better ranking via keyword + semantic")
print("   - Filtered: Adds business logic (time, type, tags)")

2025-11-27 10:15:00.839 | INFO     | app.database.qdrant_client:search:174 - Searching feedprism_emails (limit=3)
2025-11-27 10:15:00.859 | SUCCESS  | app.database.qdrant_client:search:193 - Found 3 results


🔍 Query: 'transformers'


1️⃣ Dense-Only Search (Phase 1 style):
   1. Machine Learning Workshop: Deep Dive into Transformers (score: 0.653)
   2. The Future of AI Agents (score: 0.216)
   3. Data Science Conference 2025 (score: 0.166)

2️⃣ Hybrid Search (Dense + Sparse + RRF):
   1. Machine Learning Workshop: Deep Dive into Transformers
   2. The Future of AI Agents
   3. Data Science Conference 2025

3️⃣ Hybrid + Time Filter (Upcoming Events):
   1. Machine Learning Workshop: Deep Dive into Transfor (2025-12-02)
   2. Web Development Bootcamp: React and Node.js (2025-12-12)


💡 Key Insights:
   - Dense search: Good semantic understanding
   - Hybrid: Better ranking via keyword + semantic
   - Filtered: Adds business logic (time, type, tags)


## 📊 Summary: What We Built in Phase 3

### 1. Sparse Vector Generation
- **File:** `app/utils/sparse_vector.py`
- **Function:** `create_sparse_vector(text)`
- **Output:** `{"indices": [...], "values": [...]}`
- **Uses:** Simple hash-based word indexing with term frequency

### 2. Hybrid Search
- **Method:** `qdrant.hybrid_search(query_vec, query_text, limit)`
- **Steps:**
  1. Dense search → get ranked results
  2. Sparse search → get ranked results
  3. RRF fusion → combine rankings
  4. Return top N merged results

### 3. Payload Filtering
- **Method:** `qdrant.search_with_filters(query_vec, content_type, date_range, tags, limit)`
- **Supports:**
  - Content type matching
  - Date range filtering (numeric)
  - Tag matching (multiple)

### 4. Time-Aware Helpers
- **Upcoming events:** `qdrant.search_upcoming_events(query_vec, days_ahead)`
- **Recent blogs:** `qdrant.search_recent_blogs(query_vec, days_back)`
- **Key:** Automatic timestamp conversion for Range filters

### 5. Data Model Changes
- **Dual vectors:** Points now have `""` (dense) + `"keywords"` (sparse)
- **Timestamp storage:** `start_date` as float for filtering
- **ISO string storage:** `start_date_iso` for display

---

## 🎯 Key Takeaways

1. **Hybrid > Dense alone:** Combining semantic + keyword search improves relevance
2. **RRF is simple:** No score normalization needed, just rank-based fusion
3. **Timestamps for dates:** Qdrant Range filter requires numeric values
4. **Filtering is powerful:** Combine semantic search with business logic (time, type)
5. **Dual storage pattern:** Store both machine-friendly (timestamp) and human-friendly (ISO) formats

---

## 🚀 Next Steps

**Phase 4: Qdrant Enhancements**
- Quantization for memory efficiency
- HNSW tuning for faster search
- Sharding for scale

**Try it yourself:**
- Modify the sparse vector algorithm (try TF-IDF weighting)
- Experiment with different RRF k values
- Add more filter types (author, source, sentiment)

---

## 📚 Further Reading

- [Qdrant Sparse Vectors Docs](https://qdrant.tech/documentation/concepts/vectors/#sparse-vectors)
- [Reciprocal Rank Fusion Paper](https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf)
- [BM25 Algorithm Explained](https://en.wikipedia.org/wiki/Okapi_BM25)